In [1]:
from crosscheck.crosscheck import crosscheck

In [2]:
groups = [
    [
        'https://doi.org/10.3389/fphys.2012.00450'
    ],
    [
        'https://doi.org/10.1016/j.jneumeth.2007.03.024'
    ]
]

In [4]:
results, seed = crosscheck(groups)

In [6]:
print('Papers found:')

for paper_id in results:
    print(f'https://semanticscholar.org/paper/{paper_id}')

Papers found:
https://semanticscholar.org/paper/b3f1500b2874ea55663387358055696f77ff8003
https://semanticscholar.org/paper/fe8ba37cfb9c6dbec8c604bcef431cdcb46a4cbc
https://semanticscholar.org/paper/02f79baa527f4df4fcc745671ae193eca8601d8b
https://semanticscholar.org/paper/05a361b7cbccd1a7704e9fd0d2a09abb059a469c
https://semanticscholar.org/paper/1917b2b1e66c3b5004ab27c82f3e5cb359c12fcc
https://semanticscholar.org/paper/d8824c03abe6c907187e0e89d71d94c40cfabf42
https://semanticscholar.org/paper/ca23405e20c0ca96317eacbc5bd4d277078a36cd
https://semanticscholar.org/paper/e6b7570039ba050c098da966a29a84c861c0f416
https://semanticscholar.org/paper/5799f76319a3e581e5ce90a8a75693a0bb760e52
https://semanticscholar.org/paper/186a73d66075cf2a578943f1a612912188a1a094
https://semanticscholar.org/paper/e0027fb6b23ff6e3fef017e5c4240c3aa2b6eed7
https://semanticscholar.org/paper/4c590ba12f3b779e363f7306bc90bb3e020609c9
https://semanticscholar.org/paper/5594a7a4ec9e054ceddc21ac5c2ec1e6e35b43f9
https://sem